In [1]:
import numpy as np
import os
import pandas as pd

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'         )
dirIntermediate = os.path.join(dirWork, r'intermediate'  )
dirResults      = os.path.join(dirWork, r'results'       )
dirParams       = os.path.join(dirWork, r'params'        )
dirWebApp       = os.path.join(dirResults, r'webapp_data')

In [4]:
#intermediate output
dfFactors = pd.read_csv(os.path.join(dirIntermediate,'CCS_Factors_AllGroupings.csv'))
dfSegFac  = pd.read_csv(os.path.join(dirIntermediate, 'SegmentsMatchedToFactorGroups.csv'))
dfSegFacWithTrk = pd.read_csv(os.path.join(dirIntermediate, r'SegmentTruckFactors.csv'))

#Start with UDOT Base
#sdfSegments = pd.DataFrame.spatial.from_featureclass(os.path.join(dirWork,  r"arcgis\UDOT-Segments.gdb\Master_Segs_20210811"))

#Start with Modified Base - and strip down of factor fields
sdfSegments = pd.DataFrame.spatial.from_featureclass(r"C:\Users\jlillywhite\Downloads\Segments_IR_20230912\Segments_IR_20230912.shp")
sdfSegments = sdfSegments.drop(columns=['FAC_MON'  ,'FAC_TUE'  ,'FAC_WED'  ,'FAC_THU'  ,'FAC_FRI'  ,'FAC_SAT'  ,'FAC_SUN'  ,'FAC_WDAVG','FAC_WEAVG','FAC_WEMAX','FAC_JAN'  ,'FAC_FEB'  ,'FAC_MAR'  ,'FAC_APR'  ,'FAC_MAY'  ,'FAC_JUN'  ,'FAC_JUL'  ,'FAC_AUG'  ,'FAC_SEP'  ,'FAC_OCT'  ,'FAC_NOV'  ,'FAC_DEC'  ,'FAC_WIN'   ,'FAC_SPR'   ,'FAC_SUM'   ,'FAC_FAL'  ,'FAC_MAXMO','FAC_MAX'  ,'FACMANADJ','CUTRUCKS','SUTRUCKS'])


In [5]:
dfSegFac

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP
0,0006_000.0,415,1,Arterial,Statewide,WD0
1,0006_000.0,415,1,Arterial,Statewide,SNP
2,0006_000.0,415,1,Arterial,Statewide,COP
3,0006_000.7,415,1,Arterial,Statewide,WD0
4,0006_000.7,415,1,Arterial,Statewide,SNP
...,...,...,...,...,...,...
25693,0056_060.4,13242,3,Arterial,Statewide,SNT
25694,0056_060.4,13242,3,Arterial,Statewide,COT
25695,1760_000.0,713,3,Arterial,Statewide,WD2
25696,1760_000.0,713,3,Arterial,Statewide,SNT


In [6]:
#parameters
prmInterpolate = pd.read_csv(os.path.join(dirParams,'segment_factors_interpolate.csv'))

In [7]:
dFactorGroupsToPublish = {
                           'SEASONGROUP': ['Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'Year'     ,'M01-Jan'  ,'M02-Feb'  ,'M03-Mar'  ,'M04-Apr'  ,'M05-May'  ,'M06-Jun'  ,'M07-Jul'  ,'M08-Aug'  ,'M09-Sep'  ,'M10-Oct'  ,'M11-Nov'  ,'M12-Dec'  ,'S01-Winter','S02-Spring','S03-Summer','S04-Fall' ,'Year'     ,'Year'     ],
                           'YEARGROUP'  : ['2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019','2015-2019' ,'2015-2019' ,'2015-2019' ,'2015-2019','2015-2019','2015-2019'],
                           'FACTORNAME' : ['FAC_MON'  ,'FAC_TUE'  ,'FAC_WED'  ,'FAC_THU'  ,'FAC_FRI'  ,'FAC_SAT'  ,'FAC_SUN'  ,'FAC_WDAVG','FAC_WEAVG','FAC_WEMAX','FAC_JAN'  ,'FAC_FEB'  ,'FAC_MAR'  ,'FAC_APR'  ,'FAC_MAY'  ,'FAC_JUN'  ,'FAC_JUL'  ,'FAC_AUG'  ,'FAC_SEP'  ,'FAC_OCT'  ,'FAC_NOV'  ,'FAC_DEC'  ,'FAC_WIN'   ,'FAC_SPR'   ,'FAC_SUM'   ,'FAC_FAL'  ,'FAC_MAXMO','FAC_MAX'  ]
                         }

dStationGroupsToPublish = {'STATIONGROUP': ['CO0',
                                            'CO1',
                                            'CO2',
                                            'CO3',
                                            'CO7',
                                            'CO4',
                                            'CO5',
                                            'CO6',
                                            'CO8',
                                            'CO9',
                                            'COA',
                                            'COB',
                                            'COC',
                                            'COD',
                                            'COY',
                                            'COE',
                                            'COZ',
                                            'COF',
                                            'COG',
                                            'COH',
                                            'COI',
                                            'COJ',
                                            'COK',
                                            'COL',
                                            'COM',
                                            'CON',
                                            'COO',
                                            'COP',
                                            'COQ',
                                            'COR',
                                            'COS',
                                            'COT',
                                            'COU',
                                            'XX3',
                                            'COV',
                                            'COW',
                                            'COX']}

dfFactorGroupsToPublish = pd.DataFrame(dFactorGroupsToPublish)
display(dfFactorGroupsToPublish)

dfStationGroupsToPublish = pd.DataFrame(dStationGroupsToPublish)
display(dfStationGroupsToPublish)

dfToPublish = pd.DataFrame.merge(dfStationGroupsToPublish,dfFactorGroupsToPublish,how='cross')

dfFactorsToPublish = pd.DataFrame.merge(dfFactors,dfToPublish,on=('STATIONGROUP','SEASONGROUP','YEARGROUP','FACTORNAME'))
dfFactorsToPublish = dfFactorsToPublish[['STATIONGROUP','FACTORNAME','FACTOR']]
display(dfFactorsToPublish)



,SEASONGROUP,YEARGROUP,FACTORNAME
0,Year,2015-2019,FAC_MON
1,Year,2015-2019,FAC_TUE
2,Year,2015-2019,FAC_WED
3,Year,2015-2019,FAC_THU
4,Year,2015-2019,FAC_FRI
5,Year,2015-2019,FAC_SAT
6,Year,2015-2019,FAC_SUN
7,Year,2015-2019,FAC_WDAVG
8,Year,2015-2019,FAC_WEAVG
9,Year,2015-2019,FAC_WEMAX


,STATIONGROUP
0,CO0
1,CO1
2,CO2
3,CO3
4,CO7
5,CO4
6,CO5
7,CO6
8,CO8
9,CO9


,STATIONGROUP,FACTORNAME,FACTOR
0,CO0,FAC_JAN,0.320546
1,CO1,FAC_JAN,0.431890
2,CO2,FAC_JAN,1.140254
3,CO3,FAC_JAN,0.385630
4,CO4,FAC_JAN,0.751900
...,...,...,...
1031,XX3,FAC_TUE,1.146503
1032,XX3,FAC_WDAVG,1.156777
1033,XX3,FAC_WEAVG,0.631094
1034,XX3,FAC_WED,1.158757


In [8]:
#pivot for final table
dfFactorsToPublishStationGroup_pivot = dfFactorsToPublish.pivot(index=('STATIONGROUP'),columns='FACTORNAME',values='FACTOR')

dfFactorsToPublishStationGroup_pivot = dfFactorsToPublishStationGroup_pivot.reset_index()
display(dfFactorsToPublishStationGroup_pivot)

dfFactorsToPublishStationGroup_pivot.to_csv(os.path.join(dirResults, r'CCSFactors.csv'),index=False)

FACTORNAME,STATIONGROUP,FAC_APR,FAC_AUG,FAC_DEC,FAC_FAL,FAC_FEB,FAC_FRI,FAC_JAN,FAC_JUL,FAC_JUN,...,FAC_SPR,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN
0,CO0,0.552996,1.768763,0.339189,1.108475,0.339424,1.000860,0.320546,2.065370,1.891052,...,0.650077,1.908395,1.529537,0.741635,0.674389,0.715101,1.574598,0.729279,1.619659,0.333053
1,CO1,0.582760,2.106577,0.475811,0.769845,0.455530,1.231528,0.431890,2.606509,1.674926,...,0.646407,2.129338,1.038428,0.907323,0.768244,0.823068,1.256580,0.793638,1.474731,0.454411
2,CO2,0.671815,1.230627,1.090283,0.838699,1.120695,1.013876,1.140254,1.352569,1.212064,...,0.779137,1.265087,1.429561,0.801945,0.743486,0.790750,1.445970,0.826818,1.462380,1.117077
3,CO3,0.671546,1.628511,0.509105,1.160902,0.419515,1.194500,0.385630,1.867641,1.582798,...,0.708031,1.692983,1.107345,0.913602,0.808778,0.853159,1.214239,0.837098,1.321132,0.438084
4,CO4,1.067841,1.075397,0.890894,1.005847,0.835426,1.228705,0.751900,1.169202,1.125927,...,1.044572,1.123509,1.208322,1.019173,0.810915,0.905083,1.110963,0.885159,1.208322,0.826073
5,CO5,1.009896,1.096970,0.936054,1.013327,0.872103,1.177381,0.811747,1.132835,1.094698,...,1.005204,1.108168,0.938514,1.028787,0.922349,0.970222,0.995370,0.959531,1.052227,0.873301
6,CO6,1.019008,1.039100,0.989200,1.013063,0.938145,1.143674,0.893076,1.031412,1.041674,...,1.009401,1.037396,0.676965,1.087809,1.048201,1.065382,0.821146,1.060136,0.965327,0.940140
7,CO7,0.969039,1.227104,0.860512,0.984132,0.775598,1.234005,0.770658,1.313967,1.169822,...,0.976647,1.236964,1.069461,0.974003,0.854523,0.905876,1.089586,0.889103,1.109711,0.802256
8,CO8,1.013038,1.005882,0.990366,1.011307,0.978164,1.152124,0.946206,0.997661,1.017433,...,1.010123,1.006992,0.576173,1.147067,1.122332,1.134893,0.681094,1.135278,0.786016,0.971579
9,CO9,0.997301,1.204038,0.766525,0.994055,0.668825,1.099786,0.649679,1.332204,1.322203,...,1.024787,1.286148,1.005556,1.038337,0.926471,0.979650,1.026114,0.974142,1.046672,0.695010


In [9]:
dfSegFacToPublish = pd.DataFrame.merge(dfSegFac,dfFactorsToPublish,on='STATIONGROUP')
dfSegFacToPublish

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR
0,0006_000.0,415,1,Arterial,Statewide,COP,FAC_JAN,0.815954
1,0006_000.0,415,1,Arterial,Statewide,COP,FAC_FEB,0.850393
2,0006_000.0,415,1,Arterial,Statewide,COP,FAC_MAR,0.939610
3,0006_000.0,415,1,Arterial,Statewide,COP,FAC_APR,1.036664
4,0006_000.0,415,1,Arterial,Statewide,COP,FAC_MAY,1.106483
...,...,...,...,...,...,...,...,...
239843,0215_027.4,39875,2,Freeway,I215,CO8,FAC_THU,1.147067
239844,0215_027.4,39875,2,Freeway,I215,CO8,FAC_FRI,1.152124
239845,0215_027.4,39875,2,Freeway,I215,CO8,FAC_SAT,0.786016
239846,0215_027.4,39875,2,Freeway,I215,CO8,FAC_SUN,0.576173


In [10]:
#manual interpolatation

display(prmInterpolate)

dfFactorsToPublish_InterpolateFrom = pd.DataFrame.merge(prmInterpolate,dfSegFacToPublish,left_on='SEGIDFROM',right_on='SEGID')
dfFactorsToPublish_InterpolateFrom = dfFactorsToPublish_InterpolateFrom[['SEGIDTOREV','FACTORNAME','FACTOR','PORTIONFROM']]
dfFactorsToPublish_InterpolateFrom = dfFactorsToPublish_InterpolateFrom.rename(columns={'FACTOR':'FACTORFROM'})
display(dfFactorsToPublish_InterpolateFrom)

dfFactorsToPublish_InterpolateTo   = pd.DataFrame.merge(prmInterpolate,dfSegFacToPublish,left_on='SEGIDTO'  ,right_on='SEGID')
dfFactorsToPublish_InterpolateTo   = dfFactorsToPublish_InterpolateTo  [['SEGIDTOREV','FACTORNAME','FACTOR','PORTIONTO']]
dfFactorsToPublish_InterpolateTo   = dfFactorsToPublish_InterpolateTo.rename(columns={'FACTOR':'FACTORTO'})
display(dfFactorsToPublish_InterpolateTo)

dfFactorsToPublish_Interpolate = pd.DataFrame.merge(dfFactorsToPublish_InterpolateFrom,dfFactorsToPublish_InterpolateTo,on=('SEGIDTOREV','FACTORNAME'))

dfFactorsToPublish_Interpolate['PF'] = dfFactorsToPublish_Interpolate['PORTIONFROM']/(dfFactorsToPublish_Interpolate['PORTIONTO']+dfFactorsToPublish_Interpolate['PORTIONFROM'])
dfFactorsToPublish_Interpolate['PT'] = dfFactorsToPublish_Interpolate['PORTIONTO'  ]/(dfFactorsToPublish_Interpolate['PORTIONTO']+dfFactorsToPublish_Interpolate['PORTIONFROM'])
dfFactorsToPublish_Interpolate['FACTORINTERPOLATED'] = dfFactorsToPublish_Interpolate['FACTORFROM'] * dfFactorsToPublish_Interpolate['PF'] + dfFactorsToPublish_Interpolate['FACTORTO'] * dfFactorsToPublish_Interpolate['PT']
display(dfFactorsToPublish_Interpolate)

dfFactorsToPublish_Interpolate = dfFactorsToPublish_Interpolate[['SEGIDTOREV','FACTORNAME','FACTORINTERPOLATED']]
dfFactorsToPublish_Interpolate = dfFactorsToPublish_Interpolate.rename(columns={'SEGIDTOREV':'SEGID'})
dfFactorsToPublish_Interpolate

,ID,SEGIDTOREV,SEGIDFROM,PORTIONFROM,SEGIDTO,PORTIONTO
0,1,0015_225.3,0015_222.8,11,0015_261.8,1
1,2,0015_225.8,0015_222.8,11,0015_261.8,1
2,3,0015_228.1,0015_222.8,10,0015_261.8,2
3,4,0015_230.0,0015_222.8,10,0015_261.8,2
4,5,0015_233.0,0015_222.8,10,0015_261.8,2
...,...,...,...,...,...,...
64,65,0006_175.6,0006_178.8,5,0006_173.4,6
65,66,0006_174.9,0006_178.8,4,0006_173.4,7
66,67,0006_174.4,0006_178.8,3,0006_173.4,8
67,68,0006_174.0,0006_178.8,2,0006_173.4,9


,SEGIDTOREV,FACTORNAME,FACTORFROM,PORTIONFROM
0,0015_225.3,FAC_JAN,0.751900,11
1,0015_225.3,FAC_FEB,0.835426,11
2,0015_225.3,FAC_MAR,1.045369,11
3,0015_225.3,FAC_APR,1.067841,11
4,0015_225.3,FAC_MAY,1.020505,11
...,...,...,...,...
1927,0006_173.7,FAC_THU,1.002724,1
1928,0006_173.7,FAC_FRI,1.177384,1
1929,0006_173.7,FAC_SAT,1.103053,1
1930,0006_173.7,FAC_SUN,0.918320,1


,SEGIDTOREV,FACTORNAME,FACTORTO,PORTIONTO
0,0015_225.3,FAC_JAN,0.893076,1
1,0015_225.3,FAC_FEB,0.938145,1
2,0015_225.3,FAC_MAR,0.989003,1
3,0015_225.3,FAC_APR,1.019008,1
4,0015_225.3,FAC_MAY,1.020192,1
...,...,...,...,...
1927,0006_173.7,FAC_THU,1.106184,10
1928,0006_173.7,FAC_FRI,1.145015,10
1929,0006_173.7,FAC_SAT,0.887895,10
1930,0006_173.7,FAC_SUN,0.629819,10


,SEGIDTOREV,FACTORNAME,FACTORFROM,PORTIONFROM,FACTORTO,PORTIONTO,PF,PT,FACTORINTERPOLATED
0,0015_225.3,FAC_JAN,0.751900,11,0.893076,1,0.916667,0.083333,0.763664
1,0015_225.3,FAC_FEB,0.835426,11,0.938145,1,0.916667,0.083333,0.843986
2,0015_225.3,FAC_MAR,1.045369,11,0.989003,1,0.916667,0.083333,1.040672
3,0015_225.3,FAC_APR,1.067841,11,1.019008,1,0.916667,0.083333,1.063772
4,0015_225.3,FAC_MAY,1.020505,11,1.020192,1,0.916667,0.083333,1.020479
...,...,...,...,...,...,...,...,...,...
1927,0006_173.7,FAC_THU,1.002724,1,1.106184,10,0.090909,0.909091,1.096779
1928,0006_173.7,FAC_FRI,1.177384,1,1.145015,10,0.090909,0.909091,1.147957
1929,0006_173.7,FAC_SAT,1.103053,1,0.887895,10,0.090909,0.909091,0.907455
1930,0006_173.7,FAC_SUN,0.918320,1,0.629819,10,0.090909,0.909091,0.656046


,SEGID,FACTORNAME,FACTORINTERPOLATED
0,0015_225.3,FAC_JAN,0.763664
1,0015_225.3,FAC_FEB,0.843986
2,0015_225.3,FAC_MAR,1.040672
3,0015_225.3,FAC_APR,1.063772
4,0015_225.3,FAC_MAY,1.020479
...,...,...,...
1927,0006_173.7,FAC_THU,1.096779
1928,0006_173.7,FAC_FRI,1.147957
1929,0006_173.7,FAC_SAT,0.907455
1930,0006_173.7,FAC_SUN,0.656046


In [11]:
#update with interpolated values

dfSegFacToPublishWithInterpolation = pd.DataFrame.merge(dfSegFacToPublish,dfFactorsToPublish_Interpolate,on=('SEGID','FACTORNAME'),how='left')
dfSegFacToPublishWithInterpolation = dfSegFacToPublishWithInterpolation.fillna(0)
dfSegFacToPublishWithInterpolation.loc[(dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']>0),'FACTOR'] = dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']

dfSegFacToPublishWithInterpolation['FACMANADJ'] = 0
dfSegFacToPublishWithInterpolation.loc[(dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']>0),'FACMANADJ'] = 1


dfSegFacToPublishWithInterpolation['FACTOR'] = round(dfSegFacToPublishWithInterpolation['FACTOR'],4)

display(dfSegFacToPublishWithInterpolation[dfSegFacToPublishWithInterpolation['FACTORINTERPOLATED']>0])

display(dfSegFacToPublishWithInterpolation)

,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR,FACTORINTERPOLATED,FACMANADJ
49868,0006_177.5,13166,2,Expressway,Statewide,COS,FAC_JAN,0.8176,0.817631,1
49869,0006_177.5,13166,2,Expressway,Statewide,COS,FAC_FEB,0.8606,0.860587,1
49870,0006_177.5,13166,2,Expressway,Statewide,COS,FAC_MAR,0.9438,0.943836,1
49871,0006_177.5,13166,2,Expressway,Statewide,COS,FAC_APR,0.9830,0.983032,1
49872,0006_177.5,13166,2,Expressway,Statewide,COS,FAC_MAY,1.0447,1.044662,1
...,...,...,...,...,...,...,...,...,...,...
237183,WFRC_8218,0,1,Freeway,Statewide,XX3,FAC_TUE,0.9886,0.988592,1
237184,WFRC_8218,0,1,Freeway,Statewide,XX3,FAC_WDAVG,1.0222,1.022154,1
237185,WFRC_8218,0,1,Freeway,Statewide,XX3,FAC_WEAVG,0.9200,0.920031,1
237186,WFRC_8218,0,1,Freeway,Statewide,XX3,FAC_WED,1.0228,1.022814,1


,SEGID,AADT2019,AREATYPE,FTGROUP,FACGEO,STATIONGROUP,FACTORNAME,FACTOR,FACTORINTERPOLATED,FACMANADJ
0,0006_000.0,415,1,Arterial,Statewide,COP,FAC_JAN,0.8160,0.0,0
1,0006_000.0,415,1,Arterial,Statewide,COP,FAC_FEB,0.8504,0.0,0
2,0006_000.0,415,1,Arterial,Statewide,COP,FAC_MAR,0.9396,0.0,0
3,0006_000.0,415,1,Arterial,Statewide,COP,FAC_APR,1.0367,0.0,0
4,0006_000.0,415,1,Arterial,Statewide,COP,FAC_MAY,1.1065,0.0,0
...,...,...,...,...,...,...,...,...,...,...
239843,0215_027.4,39875,2,Freeway,I215,CO8,FAC_THU,1.1471,0.0,0
239844,0215_027.4,39875,2,Freeway,I215,CO8,FAC_FRI,1.1521,0.0,0
239845,0215_027.4,39875,2,Freeway,I215,CO8,FAC_SAT,0.7860,0.0,0
239846,0215_027.4,39875,2,Freeway,I215,CO8,FAC_SUN,0.5762,0.0,0


In [12]:
#pivot for final table

dfFactorsToPublish_pivot = dfSegFacToPublishWithInterpolation.pivot(index=('SEGID','STATIONGROUP','FACMANADJ'),columns='FACTORNAME',values='FACTOR')

#change maximum month to integer
dfFactorsToPublish_pivot['FAC_MAXMO'] = dfFactorsToPublish_pivot['FAC_MAXMO'].astype(int)

dfFactorsToPublish_pivot = dfFactorsToPublish_pivot.reset_index()
display(dfFactorsToPublish_pivot)

FACTORNAME,SEGID,STATIONGROUP,FACMANADJ,FAC_APR,FAC_AUG,FAC_DEC,FAC_FAL,FAC_FEB,FAC_FRI,FAC_JAN,...,FAC_SPR,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN
0,0006_000.0,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,1.0276,1.0914,0.8818,1.0120,0.9629,0.9840,0.9734,0.9773,1.0650,0.8495
1,0006_000.7,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,1.0276,1.0914,0.8818,1.0120,0.9629,0.9840,0.9734,0.9773,1.0650,0.8495
2,0006_016.0,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,1.0276,1.0914,0.8818,1.0120,0.9629,0.9840,0.9734,0.9773,1.0650,0.8495
3,0006_046.0,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,1.0276,1.0914,0.8818,1.0120,0.9629,0.9840,0.9734,0.9773,1.0650,0.8495
4,0006_060.2,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,1.0276,1.0914,0.8818,1.0120,0.9629,0.9840,0.9734,0.9773,1.0650,0.8495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8561,WFRC_8216,XX3,0,1.0139,1.0320,0.9587,1.0104,0.9640,1.1604,0.9362,...,1.0112,1.0254,0.5290,1.1651,1.1465,1.1568,0.6311,1.1588,0.7332,0.9529
8562,WFRC_8217,COS,0,1.0119,1.0724,0.9400,1.0322,0.8982,1.1559,0.8600,...,1.0108,1.0576,0.6426,1.1133,1.0744,1.0919,0.7584,1.0878,0.8742,0.8994
8563,WFRC_8218,XX3,1,0.8790,1.1541,0.9455,0.9799,0.9400,1.1363,0.9583,...,0.9252,1.1470,0.8282,1.0551,0.9886,1.0222,0.9200,1.0228,1.0118,0.9479
8564,WFRC_8219,COS,0,1.0119,1.0724,0.9400,1.0322,0.8982,1.1559,0.8600,...,1.0108,1.0576,0.6426,1.1133,1.0744,1.0919,0.7584,1.0878,0.8742,0.8994


In [13]:
display(dfSegFacWithTrk.columns)
dfFinalFactors = pd.DataFrame.merge(dfFactorsToPublish_pivot,dfSegFacWithTrk,on=('SEGID','STATIONGROUP'))
dfFinalFactors = dfFinalFactors.drop(columns='AADT2019')
dfFinalFactors

Index(['SEGID', 'AADT2019', 'AREATYPE', 'FTGROUP', 'FACGEO', 'STATIONGROUP',
       'SUTRUCKS', 'CUTRUCKS'],
      dtype='object')

,SEGID,STATIONGROUP,FACMANADJ,FAC_APR,FAC_AUG,FAC_DEC,FAC_FAL,FAC_FEB,FAC_FRI,FAC_JAN,...,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,AREATYPE,FTGROUP,FACGEO,SUTRUCKS,CUTRUCKS
0,0006_000.0,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,0.9840,0.9734,0.9773,1.0650,0.8495,1,Arterial,Statewide,0.2496,0.2324
1,0006_000.7,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,0.9840,0.9734,0.9773,1.0650,0.8495,1,Arterial,Statewide,0.2496,0.2324
2,0006_016.0,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,0.9840,0.9734,0.9773,1.0650,0.8495,1,Arterial,Statewide,0.2496,0.2324
3,0006_046.0,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,0.9840,0.9734,0.9773,1.0650,0.8495,1,Arterial,Statewide,0.1751,0.3338
4,0006_060.2,COP,0,1.0367,1.0898,0.8822,1.0315,0.8504,1.1505,0.8160,...,0.9840,0.9734,0.9773,1.0650,0.8495,1,Arterial,Statewide,0.1751,0.3338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8561,WFRC_8216,XX3,0,1.0139,1.0320,0.9587,1.0104,0.9640,1.1604,0.9362,...,1.1568,0.6311,1.1588,0.7332,0.9529,2,Freeway,Statewide,0.1392,0.0292
8562,WFRC_8217,COS,0,1.0119,1.0724,0.9400,1.0322,0.8982,1.1559,0.8600,...,1.0919,0.7584,1.0878,0.8742,0.8994,2,Arterial,Statewide,0.1278,0.0879
8563,WFRC_8218,XX3,1,0.8790,1.1541,0.9455,0.9799,0.9400,1.1363,0.9583,...,1.0222,0.9200,1.0228,1.0118,0.9479,1,Freeway,Statewide,0.1392,0.0292
8564,WFRC_8219,COS,0,1.0119,1.0724,0.9400,1.0322,0.8982,1.1559,0.8600,...,1.0919,0.7584,1.0878,0.8742,0.8994,2,Arterial,Statewide,0.1278,0.0879


In [16]:
from datetime import datetime
#get today's date in yyyymmdd format
today = datetime.today().strftime('%Y%m%d')

sdfSegmentsWithFinalFactors = pd.DataFrame.merge(sdfSegments,dfFinalFactors,on='SEGID')
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors[['SEGID','BMP','EMP','DISTANCE','FULLNAME','FNCLASS','FC_CODE','FC_GROUP','FC_NAME','CO_FIPS','CO_NAME','PLANAREA','X','Y','ONLRS','T_MODEL','USE','RT_NUM','AADT2019','AADT2018','AADT2017','AADT2016','AADT2015','AADT2014','AADT2013','AADT2012','AADT2011','AADT2010','AADT2009','AADT2008','AADT2007','AADT2006','AADT2005','AADT2004','AADT2003','AADT2002','AADT2001','AADT2000','AADT1999','AADT1998','AADT1997','AADT1996','AADT1994','AADT1993','AADT1992','AADT1991','AADT1990','AADT1989','AADT1988','AADT1987','AADT1986','AADT1985','AADT1984','AADT1983','AADT1982','AADT1981','FACGEO','STATIONGROUP','FAC_MON','FAC_TUE','FAC_WED','FAC_THU','FAC_FRI','FAC_SAT','FAC_SUN','FAC_WDAVG','FAC_WEAVG','FAC_WEMAX','FAC_JAN','FAC_FEB','FAC_MAR','FAC_APR','FAC_MAY','FAC_JUN','FAC_JUL','FAC_AUG','FAC_SEP','FAC_OCT','FAC_NOV','FAC_DEC','FAC_WIN','FAC_SPR','FAC_SUM','FAC_FAL','FAC_MAXMO','FAC_MAX','FACMANADJ','SUTRUCKS','CUTRUCKS','SHAPE']]
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.sort_values(by=['SEGID'])

#sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.fillna(0)
sdfSegmentsWithFinalFactors.loc[:, sdfSegmentsWithFinalFactors.columns != 'SHAPE'] = (
    sdfSegmentsWithFinalFactors.loc[:, sdfSegmentsWithFinalFactors.columns != 'SHAPE']
    .fillna(0)
)


#Rename STATIONGROUP to CCSGROUP
sdfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.rename(columns={"STATIONGROUP": "CCSGROUP19"})

sdfSegmentsWithFinalFactors.spatial.to_featureclass(location=os.path.join(dirResults, r'layers\Master_Segs_withFactors_' + today + '.shp'),sanitize_columns=False)
display(sdfSegmentsWithFinalFactors)

The Field type is invalid or unsupported for the operation. [AADT2019]
The Field type is invalid or unsupported for the operation. [AADT2017]
The Field type is invalid or unsupported for the operation. [AADT2007]
The Field type is invalid or unsupported for the operation. [AADT1997]
The Field type is invalid or unsupported for the operation. [AADT1986]
The Field type is invalid or unsupported for the operation. [FACMANADJ]


,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,FC_NAME,CO_FIPS,...,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,SHAPE
0,0006_000.0,0.0,0.665,0.666641,HWY 6,Arterial,3,Arterial,Other Principal Arterial,27,...,0.8495,1.0276,1.0914,1.0315,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[236177.7000000002, 4327541.25], [..."
1,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,Other Principal Arterial,27,...,0.8495,1.0276,1.0914,1.0315,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[237241.1799999997, 4327399.720000..."
2,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,Other Principal Arterial,27,...,0.8495,1.0276,1.0914,1.0315,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[261403.90000000037, 4327045.4], [..."
3,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,Other Principal Arterial,27,...,0.8495,1.0276,1.0914,1.0315,5,1.1065,0,0.1751,0.3338,"{""paths"": [[[305380.16000000015, 4325741.85], ..."
4,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,Other Principal Arterial,27,...,0.8495,1.0276,1.0914,1.0315,5,1.1065,0,0.1751,0.3338,"{""paths"": [[[324618, 4337936.9], [324647.40000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8533,WFRC_8216,0.0,0.0,1.680911,,Arterial,20,Freeway,Proposed Freeway and Expressway,11,...,0.9529,1.0112,1.0254,1.0104,8,1.0414,0,0.1392,0.0292,"{""paths"": [[[409266.41110000014, 4549223.74059..."
8534,WFRC_8217,0.0,0.0,0.404295,,Arterial,50,Arterial,Proposed Major Collector,35,...,0.8994,1.0108,1.0576,1.0322,8,1.0724,0,0.1278,0.0879,"{""paths"": [[[417222.4900000002, 4482555.470000..."
8535,WFRC_8218,0.0,0.0,0.575753,,Freeway,1,Freeway,Interstate,35,...,0.9479,0.9252,1.1470,0.9799,7,1.1808,1,0.1392,0.0292,"{""paths"": [[[433362, 4507122.73], [433261.4064..."
8536,WFRC_8219,0.0,0.0,1.154153,,Arterial,50,Arterial,Proposed Major Collector,11,...,0.8994,1.0108,1.0576,1.0322,8,1.0724,0,0.1278,0.0879,"{""paths"": [[[419504.53110000025, 4554272.0693]..."


In [17]:
dfFinalFactors.columns

Index(['SEGID', 'STATIONGROUP', 'FACMANADJ', 'FAC_APR', 'FAC_AUG', 'FAC_DEC',
       'FAC_FAL', 'FAC_FEB', 'FAC_FRI', 'FAC_JAN', 'FAC_JUL', 'FAC_JUN',
       'FAC_MAR', 'FAC_MAX', 'FAC_MAXMO', 'FAC_MAY', 'FAC_MON', 'FAC_NOV',
       'FAC_OCT', 'FAC_SAT', 'FAC_SEP', 'FAC_SPR', 'FAC_SUM', 'FAC_SUN',
       'FAC_THU', 'FAC_TUE', 'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WED', 'FAC_WEMAX',
       'FAC_WIN', 'AREATYPE', 'FTGROUP', 'FACGEO', 'SUTRUCKS', 'CUTRUCKS'],
      dtype='object')

In [18]:
#export table to csv

dfSegmentsWithFinalFactors = sdfSegmentsWithFinalFactors.drop(columns=['SHAPE'])
dfSegmentsWithFinalFactors.to_csv(os.path.join(dirResults, r'Master_Segs_withFactors_' + today + '.csv'),index=False)

In [19]:
os.path.join(dirResults, r'Master_Segs_withFactors_' + today + '.csv')

'c:\\Users\\jlillywhite\\Projects\\GitHub\\UDOT-Segment-Factors\\results\\Master_Segs_withFactors_20241218.csv'

In [20]:
sdfSegmentsWithFinalFactors.columns

Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'FULLNAME', 'FNCLASS', 'FC_CODE',
       'FC_GROUP', 'FC_NAME', 'CO_FIPS', 'CO_NAME', 'PLANAREA', 'X', 'Y',
       'ONLRS', 'T_MODEL', 'USE', 'RT_NUM', 'AADT2019', 'AADT2018', 'AADT2017',
       'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011',
       'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005',
       'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999',
       'AADT1998', 'AADT1997', 'AADT1996', 'AADT1994', 'AADT1993', 'AADT1992',
       'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987', 'AADT1986',
       'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981', 'FACGEO',
       'CCSGROUP19', 'FAC_MON', 'FAC_TUE', 'FAC_WED', 'FAC_THU', 'FAC_FRI',
       'FAC_SAT', 'FAC_SUN', 'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WEMAX', 'FAC_JAN',
       'FAC_FEB', 'FAC_MAR', 'FAC_APR', 'FAC_MAY', 'FAC_JUN', 'FAC_JUL',
       'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC', 'FAC_WIN

In [21]:
sdfSegmentsWithFinalFactors['CCSGROUP19']

0       COP
1       COP
2       COP
3       COP
4       COP
       ... 
8533    XX3
8534    COS
8535    XX3
8536    COS
8537    COT
Name: CCSGROUP19, Length: 8566, dtype: object